In [1]:
%cd /hdd/yuchen

/hdd/yuchen


In [2]:
from PIL import Image
import torch
from torch import nn, optim
import glob
import os
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, BatchSampler
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm
import random
import nltk, re, string, collections
import scipy 
import IProgress 
from ipywidgets import FloatProgress
from nilearn import connectome
from sklearn.preprocessing import Normalizer, OrdinalEncoder, OneHotEncoder, StandardScaler
from torch.nn import Linear, ReLU, Dropout
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, TopKPooling, BatchNorm, global_mean_pool
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from matplotlib import pyplot as plt

%matplotlib inline

In [3]:
seed = 42
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True
torch.cuda.empty_cache()

In [4]:
save_dir = '/hdd/yuchen/cs292/test/nsdfeat/init_latent/'
files = glob.glob(save_dir + '*')
t = [int(f.split('/')[-1].split('.')[0]) for f in files]
t.sort()
feats_te_avg = []
for file_idx in t: feats_te_avg.append(np.load(f'{save_dir}{file_idx:06}.npy'))
feats_te_avg = np.array(feats_te_avg)

save_dir = '/hdd/yuchen/cs292/train/nsdfeat/init_latent/'
files = glob.glob(save_dir + '*')
t = [int(f.split('/')[-1].split('.')[0]) for f in files]
t.sort()
feats_tr = []
for file_idx in t: feats_tr.append(np.load(f'{save_dir}{file_idx:06}.npy'))
feats_tr = np.array(feats_tr)

In [5]:
from os.path import join as pjoin

betas_csv_dir = '/hdd/yuchen/cs292/fmri/betas_csv/'
sub = '01'
    
data_file = pjoin(betas_csv_dir, f'sub-{sub}_ResponseData.h5')
responses = pd.read_hdf(data_file)
vox_f = pjoin(betas_csv_dir, f'sub-{sub}_VoxelMetadata.csv')
voxdata = pd.read_csv(vox_f)
stim_f = pjoin(betas_csv_dir, f'sub-{sub}_StimulusMetadata.csv')
stimdata = pd.read_csv(stim_f)

train_idx = stimdata[stimdata['trial_type'] == 'train'].index.tolist()
test_idx = stimdata[stimdata['trial_type'] == 'test'].index.tolist()
    
train_fmri = responses[train_idx]
test_fmri = responses[test_idx]
    
train_labels = stimdata[stimdata['trial_type'] == 'train']['stimulus']
test_labels = stimdata[stimdata['trial_type'] == 'test']['stimulus']
    
roi_idx = voxdata[(voxdata['V1'] == 1) ]['voxel_id'].tolist()
train_fmri_temp = train_fmri.iloc[roi_idx]
test_fmri_temp = test_fmri.iloc[roi_idx]

roi_neuron = {'V1': train_fmri_temp.shape[0]}

for roi in ['V2', 'V3', 'hV4']:
    roi_idx = voxdata[(voxdata[roi] == 1)]['voxel_id'].tolist()
    train_fmri_temp = pd.concat([train_fmri_temp, train_fmri.iloc[roi_idx]])
    test_fmri_temp = pd.concat([test_fmri_temp, test_fmri.iloc[roi_idx]])
    roi_neuron[roi] = train_fmri.iloc[roi_idx].shape[0]
    
train_fmri = train_fmri_temp
test_fmri = test_fmri_temp

# roi_idx = voxdata[(voxdata['V1'] == 1) | (voxdata['V2'] == 1) | (voxdata['V3'] == 1) | (voxdata['hV4'] == 1) ]['voxel_id'].tolist()
# train_fmri = train_fmri.iloc[roi_idx]
# test_fmri = test_fmri.iloc[roi_idx]

test = stimdata[stimdata['trial_type'] == 'test']
test_fmri_avg = []
test_labels = test['stimulus'].unique()
for stim in test_labels:
    repeated_trial = test[test['stimulus'] == stim].index.tolist()
    test_fmri_avg.append(test_fmri[repeated_trial].mean(axis=1).to_numpy())
test_fmri_avg = np.array(test_fmri_avg)

del responses, voxdata, stimdata
# train_fmri, test_fmri =  train_fmri.to_numpy(), test_fmri.to_numpy()
train_img_label_all, test_img_label_all = train_labels.tolist(), test_labels.tolist()

betas_tr = train_fmri.T
betas_te_avg = test_fmri_avg

In [6]:
roi_neuron

{'V1': 1049, 'V2': 774, 'V3': 762, 'hV4': 613}

In [7]:
test_fmri_avg.shape

(100, 3198)

In [8]:
print('feats_tr', feats_tr.shape)
print('betas_tr', betas_tr.shape)

print('feats_te_avg', feats_te_avg.shape)
print('betas_te_avg', betas_te_avg.shape)

feats_tr (8640, 6400)
betas_tr (8640, 3198)
feats_te_avg (100, 6400)
betas_te_avg (100, 3198)


In [9]:
betas_tr = np.array(betas_tr)
betas_te_avg_ = np.array(betas_te_avg)

betas_tr_ = (betas_tr - betas_tr.mean()) / betas_tr.std()
betas_te_avg_ = (betas_te_avg - betas_te_avg.mean()) / betas_te_avg.std()

In [10]:
betas_tr_.shape

(8640, 3198)

## goal: 1.8787439

In [11]:
import argparse, os
import numpy as np
from himalaya.backend import set_backend
from himalaya.ridge import RidgeCV
from himalaya.scoring import correlation_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
alpha = [0.000001,0.00001,0.0001,0.001,0.01, 0.1, 1]

ridge = RidgeCV(alphas=alpha)

preprocess_pipeline = make_pipeline(
        StandardScaler(with_mean=True, with_std=True),
    )
pipeline = make_pipeline(
        preprocess_pipeline,
        ridge,
)    

In [12]:
X = betas_tr
Y = feats_tr
X_te = betas_te_avg
pipeline.fit(X, Y)
scores = pipeline.predict(X_te)
from sklearn.metrics import mean_squared_error
mean_squared_error(scores,feats_te_avg)

0.74321836

In [14]:
np.save('/hdd/yuchen/things_baseline_latent_init.npy',scores)

## gcn best: 0.55675

In [11]:
betas_tr_temp = betas_tr_[:, :roi_neuron['V1']]
num_feature = 1375
zeros = np.zeros((betas_tr_temp.shape[0], num_feature))
zeros[:, :betas_tr_temp.shape[-1]] = betas_tr_temp
betas_tr_temp = zeros  # (8640, 1573)

for roi in [2,3]:
    temp = betas_tr_[:, sum(list(roi_neuron.values())[:roi-1]):sum(list(roi_neuron.values())[:roi])]
    if roi == 3: temp = betas_tr_[:, sum(list(roi_neuron.values())[:roi-1]):]
    zeros = np.zeros((betas_tr_.shape[0], num_feature))
    zeros[:, :temp.shape[-1]] = temp
    betas_tr_temp = np.hstack([betas_tr_temp, zeros])

betas_te_avg_temp = betas_te_avg_[:, :roi_neuron['V1']]
zeros = np.zeros((betas_te_avg_temp.shape[0], num_feature))
zeros[:, :betas_te_avg_temp.shape[-1]] = betas_te_avg_temp
betas_te_avg_temp = zeros
# for roi in [2,3,4]:
for roi in [2,3]:
    temp = betas_te_avg_[:, sum(list(roi_neuron.values())[:roi-1]):sum(list(roi_neuron.values())[:roi])]
    if roi == 3: temp = betas_te_avg_[:, sum(list(roi_neuron.values())[:roi-1]):]
    zeros = np.zeros((betas_te_avg_.shape[0], num_feature))
    zeros[:, :temp.shape[-1]] = temp
    betas_te_avg_temp = np.hstack([betas_te_avg_temp, zeros])

In [12]:
betas_tr_ = betas_tr_temp
betas_te_avg_ = betas_te_avg_temp

In [13]:
num_roi = 3

num_neurons = betas_tr_.shape[-1]
num_pad = (num_neurons // num_roi + 1) * num_roi - num_neurons

betas_tr = np.pad(betas_tr_, ((0, 0), (0, num_pad)), 'constant', constant_values=0)
betas_te_avg = np.pad(betas_te_avg_, ((0, 0), (0, num_pad)), 'constant', constant_values=0)

In [14]:
num_neurons_each_roi = int(betas_te_avg.shape[-1]/num_roi)

conn_measure = connectome.ConnectivityMeasure(kind='correlation')

function_con = betas_tr.reshape(-1, num_roi, num_neurons_each_roi)

connectivity = conn_measure.fit_transform([np.mean(function_con, axis=-1)])
for i in range(len(connectivity[0])):
    connectivity[0][i, i] = 0  # setting the i-th row to zero

/hdd/yuchen/anaconda3/envs/ldm/lib/python3.8/site-packages/nilearn/connectome/connectivity_matrices.py:509: FutureWarning: The default strategy for standardize is currently 'zscore' which incorrectly uses population std to calculate sample zscores. The new strategy 'zscore_sample' corrects this behavior by using the sample std. In release 0.13, the default strategy will be replaced by the new strategy and the 'zscore' option will be removed. Please use 'zscore_sample' instead.
  covariances_std = [


In [15]:
connectivity.shape

(1, 3, 3)

In [16]:
adjmatrix = np.where(abs(connectivity[0]) > 0.5)
edge_index = np.array(adjmatrix)  # edge index matrix of shape [2, num_edges]

edge_index.shape

(2, 6)

In [17]:
edge_attr = []
for idx in range(edge_index.shape[1]):
    edge_attr.append(connectivity[0][edge_index[0,idx], edge_index[1,idx]])
edge_attr = np.array(edge_attr)
edge_attr.shape

(6,)

In [18]:
edge_index

array([[0, 0, 1, 1, 2, 2],
       [1, 2, 0, 2, 0, 1]])

In [19]:
edge_index_temp = []
edge_attr_temp = []
for i in range(edge_attr.shape[0]):
    edge = edge_index.T[i]
    if edge[0] < edge[1]:
        edge_index_temp.append(edge)
        edge_attr_temp.append(edge_attr[i])
edge_attr = np.array(edge_attr_temp)
edge_index = np.array(edge_index_temp).T



In [20]:
num_trial_tr, num_trials_te = feats_tr.shape[0], feats_te_avg.shape[0]

train_dataset = []

for i in range(num_trial_tr):
    train_dataset.append(Data(x=torch.tensor(betas_tr.reshape(-1, num_roi, num_neurons_each_roi)[i], dtype=torch.float), 
            edge_index=torch.tensor(edge_index, dtype=torch.long), 
            edge_attr=torch.tensor(edge_attr, dtype=torch.float),
            y=torch.tensor(feats_tr[i], dtype=torch.float)
            ))
    
test_dataset = []
for i in range(num_trials_te):
    test_dataset.append(Data(x=torch.tensor(betas_te_avg.reshape(-1, num_roi, num_neurons_each_roi)[i], dtype=torch.float), 
            edge_index=torch.tensor(edge_index, dtype=torch.long), 
            edge_attr=torch.tensor(edge_attr, dtype=torch.float),
            y=torch.tensor(feats_te_avg[i], dtype=torch.float)
            ))

In [21]:
len(train_dataset), len(test_dataset)

(8640, 100)

In [22]:
train_dataset[0], test_dataset[0]

(Data(x=[3, 1376], edge_index=[2, 3], edge_attr=[3], y=[6400]),
 Data(x=[3, 1376], edge_index=[2, 3], edge_attr=[3], y=[6400]))

In [23]:
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [43]:
from torch_geometric.nn import GCNConv, TopKPooling, BatchNorm,ChebConv, SAGEConv,GATConv
from torch_geometric.nn import global_mean_pool, global_max_pool,global_add_pool

class GNN(torch.nn.Module):
    def __init__(self, num_neurons_each_roi):
        super(GNN, self).__init__()
        
        self.conv1 = ChebConv(num_neurons_each_roi, 128, K=4)

        self.conv = ChebConv(128, 128, K=3)
        self.bn = BatchNorm(128)

        nnconv_channel = 2
        self.nnconv = nn.Sequential(
            nn.Conv1d(1, nnconv_channel, kernel_size=1),
            nn.ReLU(),
        )
        self.net = nn.Sequential(
            nn.Conv1d(nnconv_channel, nnconv_channel, kernel_size=3, padding = 1),
            nn.ReLU(),
        )
        self.downsample = nn.Conv1d(nnconv_channel, 1, kernel_size=1)

        self.fc = nn.Sequential(
            nn.Linear(128, 512),
            nn.ReLU(),
            nn.Linear(512, 6400)
        )

    def forward(self, data):
        x, edge_index, batch,edge_attr = data.x, data.edge_index, data.batch, data.edge_attr
        batch_size = batch.max() + 1

        x = F.relu(self.conv1(x, edge_index, edge_attr))
        
        for i in range(1):
            a = self.conv(x, edge_index, edge_attr)
            a = self.bn(a)
            x = x + F.leaky_relu(a)
        
        x = global_mean_pool(x, batch) # print(x.shape) torch.Size([64, 256])

        x = x.unsqueeze(1)
        x = self.nnconv(x)
        for j in range(1):
            x = x + self.net(x)
        x = self.downsample(x)
        x = x.flatten(start_dim=1)
        x = self.fc(x)
        
        return x

In [44]:
device = 'cuda:0'
model = GNN(num_neurons_each_roi=num_neurons_each_roi).to(device)

loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0015)

num_epochs = 100
patience = 5
ct = 0
best_loss_val = np.inf

for epoch in range(num_epochs):
    total_loss_train = 0
    model.train()
    for batch in train_dataloader:
        batch = batch.to(device)
        batch_num = batch.batch.max() + 1
        feats = batch['y'].reshape(batch_num,-1).to(device)
        pred = model(batch)
        
        loss = loss_fn(pred, feats)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss_train += loss.item()
    print('[epoch {}] train loss {}'.format(epoch, round(total_loss_train/len(train_dataloader), 5)), end = ' ')

    model.eval()
    total_loss_val = 0
    with torch.no_grad():
        for batch in test_dataloader:
            batch = batch.to(device)
            batch_num = batch.batch.max() + 1
            feats = batch['y'].reshape(batch_num,-1).to(device)
            pred = model(batch)
            loss = loss_fn(pred, feats)
            total_loss_val += loss.item()
        total_loss_val = total_loss_val/len(test_dataloader)
        print('val loss {}'.format(round(total_loss_val, 5)))
        
        if total_loss_val < best_loss_val:
            ct = 0
            best_loss_val = total_loss_val
            torch.save(model.state_dict(), '/hdd/yuchen/modelweights_things_visual_gcn_roi.pth')
        else:
            ct += 1
            if ct > patience:
                print('end')
                break

[epoch 0] train loss 0.72386 val loss 0.71847
[epoch 1] train loss 0.71281 val loss 0.70301
[epoch 2] train loss 0.70046 val loss 0.69713
[epoch 3] train loss 0.68225 val loss 0.69702
[epoch 4] train loss 0.66077 val loss 0.68736
[epoch 5] train loss 0.63916 val loss 0.71032
[epoch 6] train loss 0.62045 val loss 0.70825
[epoch 7] train loss 0.60451 val loss 0.69806
[epoch 8] train loss 0.5905 val loss 0.7187
[epoch 9] train loss 0.57821 val loss 0.73498
[epoch 10] train loss 0.56881 val loss 0.72134
end


In [59]:
best_loss_val

0.6873591840267181

In [60]:
device = 'cuda:0'
model = GNN(num_neurons_each_roi=num_neurons_each_roi).to(device)
model.load_state_dict(torch.load('/hdd/yuchen/modelweights_things_visual_gcn_roi.pth'))

<All keys matched successfully>

In [61]:
model.eval()
lst = []
lst_gt = []

with torch.no_grad():
    for batch in test_dataloader:
        batch = batch.to(device)
        batch_num = batch.batch.max() + 1
        pred = model(batch)
        batch_num = batch.batch.max() + 1
        feats = batch['y'].reshape(batch_num,-1).to(device)
        
        lst.append(pred.cpu().detach().numpy())
        lst_gt.append(feats.cpu().detach().numpy())

In [62]:
lst = np.vstack(lst)
lst_gt = np.vstack(lst_gt)

In [63]:
from sklearn.metrics import mean_squared_error
mean_squared_error(lst,lst_gt)

0.68060076

In [64]:
np.save('/hdd/yuchen/things_temp_lst_latent_init.npy', lst)

In [10]:
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np

class CustomDataset(Dataset):
    def __init__(self, betas, feats):
        self.betas = torch.tensor(betas, dtype=torch.float32)
        self.feats = torch.tensor(feats, dtype=torch.float32)

    def __len__(self):
        return len(self.betas)

    def __getitem__(self, idx):
        return {'betas': self.betas[idx], 'feats': self.feats[idx]}

train_dataset = CustomDataset(betas_tr_, feats_tr)
test_dataset = CustomDataset(betas_te_avg_, feats_te_avg)

In [11]:
train_dataset[0]['betas'].shape, train_dataset[0]['feats'].shape

(torch.Size([3198]), torch.Size([6400]))

In [12]:
batch_size = 256  
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

for batch in train_dataloader:
    break
num_neurons = batch['betas'].shape[-1]
num_features = batch['feats'].shape[-1]

In [13]:
from torch import nn

class NeuralNetwork(nn.Module):
    def __init__(self, num_neurons, num_features):
        super(NeuralNetwork, self).__init__()
        
        self.num_neurons = num_neurons
        channel = 64
        self.conv1d = nn.Conv1d(1, channel, kernel_size=1)
        self.relu = nn.ReLU()
        self.net = nn.Sequential(
            nn.Conv1d(channel, channel, kernel_size=11, padding = 5),
            nn.ReLU(),
            nn.Conv1d(channel, channel, kernel_size=11, padding = 5),
            nn.ReLU(),
        )
        self.downsample = nn.Conv1d(channel, 1, kernel_size=1)

        self.fc = nn.Sequential(
            nn.Linear(num_neurons, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 6400)
        )

    def forward(self, x):
        x = self.conv1d(x)
        for i in range(2):
            x = x + self.net(x)
        x = self.downsample(x)
        x = x.reshape(-1, self.num_neurons)
        x = self.fc(x)
        return x

In [14]:
device = 'cuda:0'

model = NeuralNetwork(num_neurons = num_neurons, num_features = num_features).to(device)

loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)

num_epochs = 100
patience = 5
ct = 0
best_loss_val = np.inf

for epoch in range(num_epochs):
    total_loss_train = 0
    model.train()
    for batch in train_dataloader:
        betas = batch['betas'].to(device).unsqueeze(1)
        feats = batch['feats'].to(device)
        
        pred = model(betas)
        loss = loss_fn(pred, feats)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss_train += loss.item()
    print('[epoch {}] train loss {}'.format(epoch, round(total_loss_train/len(train_dataloader), 5)), end = ' ')

    model.eval()
    total_loss_val = 0
    with torch.no_grad():
        for batch in test_dataloader:
            betas = batch['betas'].to(device).unsqueeze(1)
            feats = batch['feats'].to(device)
            pred = model(betas)
            loss = loss_fn(pred, feats)
            total_loss_val += loss.item()
        total_loss_val = total_loss_val/len(test_dataloader)
        print('val loss {}'.format(round(total_loss_val, 5)))
        
        if total_loss_val < best_loss_val:
            ct = 0
            best_loss_val = total_loss_val
            torch.save(model.state_dict(), '/hdd/yuchen/modelweights_visual_things_textnn.pth')
        else:
            ct += 1
            if ct > patience:
                print('end')
                break

[epoch 0] train loss 0.73409 val loss 0.70004
[epoch 1] train loss 0.71778 val loss 0.69203
[epoch 2] train loss 0.70781 val loss 0.69262
[epoch 3] train loss 0.69221 val loss 0.69515
[epoch 4] train loss 0.66643 val loss 0.69906
[epoch 5] train loss 0.64036 val loss 0.70593
[epoch 6] train loss 0.61582 val loss 0.74341
[epoch 7] train loss 0.59286 val loss 0.75947
end


In [15]:
best_loss_val

0.6920348405838013

In [16]:
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np

class CustomDataset(Dataset):
    def __init__(self, betas, feats):
        self.betas = torch.tensor(betas, dtype=torch.float32)
        self.feats = torch.tensor(feats, dtype=torch.float32)

    def __len__(self):
        return len(self.betas)

    def __getitem__(self, idx):
        return {'betas': self.betas[idx], 'feats': self.feats[idx]}

train_dataset = CustomDataset(betas_tr_, feats_tr)
test_dataset = CustomDataset(betas_te_avg_, feats_te_avg)

In [17]:
batch_size = 256  
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

for batch in train_dataloader:
    break
num_neurons = batch['betas'].shape[-1]
num_features = batch['feats'].shape[-1]

In [18]:
import torch
from torch import nn
from torch.nn import functional as F

class FBL(nn.Module):
    def __init__(self, in_features, out_features):
        super(FBL, self).__init__()
        self.fc = nn.Linear(in_features, out_features)
        self.bn = nn.BatchNorm1d(out_features)
        self.act = nn.LeakyReLU()

    def forward(self, x):
        x = self.fc(x)
        x = self.bn(x)
        return self.act(x)

class Encoder(nn.Module):
    def __init__(self, input_dim, hidden_dims, z_dim):
        super(Encoder, self).__init__()
        self.ln1 = nn.Linear(input_dim, 512)
        self.relu = nn.LeakyReLU()
        self.ln2 = nn.Linear(512, 256)
        self.fc_mu = nn.Linear(256, z_dim)
        self.fc_log_var = nn.Linear(256, z_dim)

    def forward(self, x):
        x = self.ln1(x)
        x = self.relu(x)
        x = self.ln2(x)
        mu = self.fc_mu(x)
        log_var = self.fc_log_var(x)
        return mu, log_var

class Decoder(nn.Module):
    def __init__(self, z_dim, hidden_dims, output_dim):
        super(Decoder, self).__init__()
        self.fbl1 = FBL(z_dim, hidden_dims)
        self.fbl2 = FBL(hidden_dims, hidden_dims)
        self.fbl3 = FBL(hidden_dims, hidden_dims)
        self.final_fc = nn.Linear(hidden_dims, output_dim)

    def forward(self, z):
        z = self.fbl1(z)
        for i in range(1):
            z = self.fbl2(z)
        z = self.fbl3(z)
        return self.final_fc(z)

class VAE(nn.Module):
    def __init__(self, fMRI_input_dim, hidden_dims, z_dim, output_dim):
        super(VAE, self).__init__()
        self.fMRI_encoder = Encoder(fMRI_input_dim, hidden_dims, z_dim)
        self.decoder = Decoder(z_dim, hidden_dims, output_dim)

    def reparameterize(self, mu, log_var):
        std = torch.exp(0.5*log_var)
        eps = torch.randn_like(std)
        return mu + eps*std

    def forward(self, fMRI):
        fMRI_mu, fMRI_log_var = self.fMRI_encoder(fMRI)
        z_fMRI = self.reparameterize(fMRI_mu, fMRI_log_var)
        fMRI_embedding = self.decoder(z_fMRI)

        return fMRI_embedding, fMRI_mu, fMRI_log_var, z_fMRI


In [20]:
device = 'cuda:0'

fMRI_input_dim = num_neurons  
hidden_dims = 4096 
z_dim = num_features  
output_dim = num_neurons  

model = VAE(fMRI_input_dim, hidden_dims, z_dim, output_dim).to(device)

loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)

num_epochs = 100
patience = 5
ct = 0
best_loss_val = np.inf

for epoch in range(num_epochs):
    total_loss_train = 0
    model.train()
    for batch in train_dataloader:
        betas = batch['betas'].to(device)
        feats = batch['feats'].to(device)
        fMRI_embedding, fMRI_mu, fMRI_log_var, z_fMRI = model(betas)
        recon_loss_fMRI = loss_fn(fMRI_embedding, betas)
        kld_fMRI = loss_fn(feats, z_fMRI)
        loss = recon_loss_fMRI + kld_fMRI
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss_train += loss.item()
    print('[epoch {}] train loss {}'.format(epoch, round(total_loss_train/len(train_dataloader), 5)), end = ' ')
        
        
    model.eval()
    total_loss_val = 0
    with torch.no_grad():
        for batch in test_dataloader:
            betas = batch['betas'].to(device)
            feats = batch['feats'].to(device)

            fMRI_embedding, fMRI_mu, fMRI_log_var, z_fMRI = model(betas)
        
            loss = loss_fn(feats, z_fMRI)
            total_loss_val += loss.item()
        
        total_loss_val = total_loss_val/len(test_dataloader)
        print('val loss {}'.format(round(total_loss_val, 5)))
        
        if total_loss_val < best_loss_val:
            ct = 0
            best_loss_val = total_loss_val
            torch.save(model.state_dict(), '/hdd/yuchen/modelweights_visual_things_vae.pth')
        else:
            ct += 1
            if ct > patience:
                print('end')
                break

[epoch 0] train loss 1.92623 val loss 0.78619
[epoch 1] train loss 1.4294 val loss 0.75204
[epoch 2] train loss 1.39404 val loss 0.73553
[epoch 3] train loss 1.3761 val loss 0.73762
[epoch 4] train loss 1.35226 val loss 0.73101
[epoch 5] train loss 1.33352 val loss 0.7342
[epoch 6] train loss 1.32215 val loss 0.72603
[epoch 7] train loss 1.30154 val loss 0.74174
[epoch 8] train loss 1.28702 val loss 0.75498
[epoch 9] train loss 1.26782 val loss 0.7313
[epoch 10] train loss 1.25246 val loss 0.74651
[epoch 11] train loss 1.24498 val loss 0.74528
[epoch 12] train loss 1.23276 val loss 0.77179
end


In [22]:
best_loss_val

0.7260345816612244

In [10]:
import argparse, os
import numpy as np
from himalaya.backend import set_backend
from himalaya.ridge import RidgeCV
from himalaya.scoring import correlation_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
alpha = [0.000001,0.00001,0.0001,0.001,0.01, 0.1, 1]

ridge = RidgeCV(alphas=alpha)

preprocess_pipeline = make_pipeline(
        StandardScaler(with_mean=True, with_std=True),
    )
pipeline = make_pipeline(
        preprocess_pipeline,
        ridge,
)    

X = betas_tr
Y = feats_tr
X_te = betas_te_avg
pipeline.fit(X, Y)
scores = pipeline.predict(X_te)
from sklearn.metrics import mean_squared_error
mean_squared_error(scores,feats_te_avg)

0.6971274161290577

In [11]:
num_roi = 4
num_neurons = betas_tr_.shape[-1]
num_pad = (num_neurons // num_roi + 1) * num_roi - num_neurons

betas_tr = np.pad(betas_tr_, ((0, 0), (0, num_pad)), 'constant', constant_values=0)
betas_tr_avg = np.pad(betas_tr_avg_, ((0, 0), (0, num_pad)), 'constant', constant_values=0)
betas_te = np.pad(betas_te_, ((0, 0), (0, num_pad)), 'constant', constant_values=0)
betas_te_avg = np.pad(betas_te_avg_, ((0, 0), (0, num_pad)), 'constant', constant_values=0)

num_neurons_each_roi = int(betas_te_avg.shape[-1]/num_roi)
conn_measure = connectome.ConnectivityMeasure(kind='correlation')
function_con = betas_tr.reshape(-1, num_roi, num_neurons_each_roi)
connectivity = conn_measure.fit_transform([np.mean(function_con, axis=-1)])
for i in range(len(connectivity[0])):
    connectivity[0][i, i] = 0  # Setting the i-th row to zero

/hdd/yuchen/anaconda3/envs/ldm/lib/python3.8/site-packages/nilearn/connectome/connectivity_matrices.py:509: FutureWarning: The default strategy for standardize is currently 'zscore' which incorrectly uses population std to calculate sample zscores. The new strategy 'zscore_sample' corrects this behavior by using the sample std. In release 0.13, the default strategy will be replaced by the new strategy and the 'zscore' option will be removed. Please use 'zscore_sample' instead.
  covariances_std = [


In [14]:
adjmatrix = np.where(abs(connectivity[0]) > 0.8)
edge_index = np.array(adjmatrix)  # Edge index matrix of shape [2, num_edges]
print(edge_index.shape)

edge_attr = []
for idx in range(edge_index.shape[1]):
    edge_attr.append(connectivity[0][edge_index[0,idx], edge_index[1,idx]])
edge_attr = np.array(edge_attr)

(2, 4)


In [15]:
num_trial_tr, num_trials_te = feats_tr.shape[0], feats_te_avg.shape[0]

train_dataset = []
for i in range(num_trial_tr):
    train_dataset.append(Data(x=torch.tensor(betas_tr.reshape(-1, num_roi, num_neurons_each_roi)[i], dtype=torch.float), 
            edge_index=torch.tensor(edge_index, dtype=torch.long), 
            edge_attr=torch.tensor(edge_attr, dtype=torch.float),
            y=torch.tensor(feats_tr[i], dtype=torch.float)
            ))
    
test_dataset = []
for i in range(num_trials_te):
    test_dataset.append(Data(x=torch.tensor(betas_te_avg.reshape(-1, num_roi, num_neurons_each_roi)[i], dtype=torch.float), 
            edge_index=torch.tensor(edge_index, dtype=torch.long), 
            edge_attr=torch.tensor(edge_attr, dtype=torch.float),
            y=torch.tensor(feats_te_avg[i], dtype=torch.float)
            ))

train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [16]:
from torch_geometric.nn import GCNConv, TopKPooling, BatchNorm,ChebConv, SAGEConv,GATConv
from torch_geometric.nn import global_mean_pool, global_max_pool,global_add_pool

class GCNWithPooling(torch.nn.Module):
    def __init__(self, num_neurons_each_roi):
        super(GCNWithPooling, self).__init__()
        
        self.conv1 = ChebConv(num_neurons_each_roi, 256, K=4)

        self.dropout = nn.Dropout(p=0.1)
        self.conv = ChebConv(256, 256, K=3)
        self.bn = BatchNorm(256)

        nnconv_channel = 16
        self.nnconv = nn.Sequential(
            nn.Conv1d(1, nnconv_channel, kernel_size=1),
            nn.ReLU(), 
        )
        self.net = nn.Sequential(
            nn.Conv1d(nnconv_channel, nnconv_channel, kernel_size=3, padding = 1),
            nn.ReLU(),
        )
        self.downsample = nn.Conv1d(nnconv_channel, 1, kernel_size=1)

        self.fc = nn.Sequential(
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, 6400)
        )

    def forward(self, data):
        x, edge_index, batch,edge_attr = data.x, data.edge_index, data.batch, data.edge_attr
        batch_size = batch.max() + 1

        x = F.relu(self.conv1(x, edge_index, edge_attr))

        for i in range(1):
            a = self.conv(x, edge_index, edge_attr)
            a = self.bn(a)
            x = x + F.relu(a)

        x = global_mean_pool(x, batch) # print(x.shape) torch.Size([64, 256])

        x = x.unsqueeze(1)
        x = self.nnconv(x)
        
        for j in range(2):
            x = x + self.net(x)
        x = self.downsample(x)
        x = x.flatten(start_dim=1)
        x = self.fc(x)
        
        return x

In [19]:
device = 'cuda:1'
model = GCNWithPooling(num_neurons_each_roi=num_neurons_each_roi).to(device)

loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.00001)

num_epochs = 100
patience = 5
ct = 0
best_loss_val = np.inf

for epoch in range(num_epochs):
    total_loss_train = 0
    model.train()
    for batch in train_dataloader:
        batch = batch.to(device)
        batch_num = batch.batch.max() + 1
        feats = batch['y'].reshape(batch_num,-1).to(device)
        pred = model(batch)
        
        loss = loss_fn(pred, feats)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss_train += loss.item()
    print('[epoch {}] train loss {}'.format(epoch, round(total_loss_train/len(train_dataloader), 5)), end = ' ')

    model.eval()
    total_loss_val = 0
    with torch.no_grad():
        for batch in test_dataloader:
            batch = batch.to(device)
            batch_num = batch.batch.max() + 1
            feats = batch['y'].reshape(batch_num,-1).to(device)
            pred = model(batch)
            loss = loss_fn(pred, feats)
            total_loss_val += loss.item()
        total_loss_val = total_loss_val/len(test_dataloader)
        print('val loss {}'.format(round(total_loss_val, 5)))
        
        if total_loss_val < best_loss_val:
            ct = 0
            best_loss_val = total_loss_val
            torch.save(model.state_dict(), '/hdd/yuchen/modelweights_visual_gcn_test.pth')
        else:
            ct += 1
            if ct > patience:
                print('end')
                break

[epoch 0] train loss 0.65145 val loss 0.62354
[epoch 1] train loss 0.64557 val loss 0.62117
[epoch 2] train loss 0.64482 val loss 0.62083
[epoch 3] train loss 0.64456 val loss 0.62064
[epoch 4] train loss 0.64436 val loss 0.62031
[epoch 5] train loss 0.64402 val loss 0.61988
[epoch 6] train loss 0.64311 val loss 0.61861
[epoch 7] train loss 0.64154 val loss 0.6169
[epoch 8] train loss 0.63933 val loss 0.61434
[epoch 9] train loss 0.63718 val loss 0.61329
[epoch 10] train loss 0.63523 val loss 0.61178
[epoch 11] train loss 0.63353 val loss 0.61143
[epoch 12] train loss 0.63188 val loss 0.61121
[epoch 13] train loss 0.63019 val loss 0.61096
[epoch 14] train loss 0.62867 val loss 0.61115
[epoch 15] train loss 0.62739 val loss 0.611
[epoch 16] train loss 0.62583 val loss 0.6112
[epoch 17] train loss 0.62411 val loss 0.61157
[epoch 18] train loss 0.6224 val loss 0.61154
[epoch 19] train loss 0.62063 val loss 0.61303
end


In [181]:
out = model(batch)
out.shape

torch.Size([16, 6400])

In [151]:
model(dataset[0]).shape

torch.Size([1480, 64])

In [146]:
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np

In [169]:
out = model(batch)
out.reshape(16, -1, 64).shape

torch.Size([16, 1480, 64])

In [140]:
train_loader = DataLoader(dataset, batch_size=1)
len(train_loader)

100

In [148]:
from torch_geometric.loader import DataLoader

In [53]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.datasets import Planetoid

# Load a dataset
dataset = Planetoid(root='/tmp/Cora', name='Cora')
data = dataset[0]

In [54]:
data

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])

In [55]:
data.num_classes

AttributeError: 'GlobalStorage' object has no attribute 'num_classes'